In [143]:
import pandas as pd
import re

In [144]:
from tabulate import tabulate

## Codigo de Entrada

In [145]:
nome_arquivo = "lights.p"
codigo_entrada = open("lib/" + nome_arquivo, "r", encoding="utf8").read()

In [146]:
print(codigo_entrada)

program lights;
uses crt;
var a,b:integer;
label 1;
begin
textbackground(white);
clrscr;
for a:=1 to 1000000 do inc b;
randomize;
1:
a:=random(4)+1;
if a=1 then textbackground(blue);
if a=2 then textbackground(red);
if a=3 then textbackground(green);
if a=4 then textbackground(yellow);
write(' ');
goto 1;
end.


## Definição de DataFrame resultante

In [147]:
rows = []
df = pd.DataFrame(rows, columns=["Tipo", "Palavra", "Linha", "Posicao_Inicio", "Posicao_Fim"])

df

,Tipo,Palavra,Linha,Posicao_Inicio,Posicao_Fim


## Procurar Palavras Reservadas no Código

In [148]:
palavras_reservadas = pd.read_csv("lib/palavras_reservadas.csv", header=None)
palavras_reservadas = list(palavras_reservadas[0])

In [149]:
tipo = "Palavras Reservadas"

In [150]:
for idx, palavra in enumerate(palavras_reservadas):
    # print(f'id = {idx}, palavra = {palavra}')
    regex = rf"(?i){palavra}"

    matches = re.finditer(regex, codigo_entrada, re.MULTILINE)

    for matchNum, match in enumerate(matches, start=1):
        
        line = codigo_entrada[0:match.end()].count('\n') + 1   
        rows.append([tipo, match.group(), line, match.start(), match.end()])

## Procurar Símbolos no Código

In [151]:
simbolos = pd.read_csv("lib/simbolos.csv", delimiter=";", header = None)
simbolos = list(simbolos[0])

In [152]:
tipo = "Símbolos"

In [153]:
for idx, palavra in enumerate(simbolos):
    # print(f'id = {idx}, palavra = {palavra}')
    regex = rf"(?i){re.escape(palavra)}"

    try:
        matches = re.finditer(regex, codigo_entrada, re.MULTILINE)
    except:
        print(f'Não foi possível a palavra: {palavra}')

    for matchNum, match in enumerate(matches, start=1):
        
        line = codigo_entrada[0:match.end()].count('\n') + 1   
        rows.append([tipo, match.group(), line, match.start(), match.end()])

## Procurar Variáveis no Código

In [154]:
tipo = "Variáveis"

In [155]:
# print(f'id = {idx}, palavra = {palavra}')
regex = rf"(?i).*:="

matches = re.finditer(regex, codigo_entrada, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    line = codigo_entrada[0:match.end()].count('\n') + 1 
    variable = match.group().split(":=")[0].split(" ")[-1]
    rows.append([tipo, variable, line, match.start(), match.end()])

## Procurar Números Inteiros no Código

In [156]:
tipo = "Números Inteiros"

In [157]:
# print(f'id = {idx}, palavra = {palavra}')
regex = r"[-+]?(?<![\d.])\d+(?![\d.])"

matches = re.finditer(regex, codigo_entrada, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    line = codigo_entrada[0:match.end()].count('\n') + 1 
    variable = match.group()
    rows.append([tipo, variable, line, match.start(), match.end()])

## Procurar Números Decimais no Código

In [158]:
tipo = "Números Decimais"

In [159]:
# print(f'id = {idx}, palavra = {palavra}')
regex = r"[-+]?[0-9]*(\.[0-9]+)"


matches = re.finditer(regex, codigo_entrada, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    line = codigo_entrada[0:match.end()].count('\n') + 1 
    variable = match.group()
    rows.append([tipo, variable, line, match.start(), match.end()])

## DataFrame resultante

In [160]:
df = pd.DataFrame(rows, columns=["Tipo", "Palavra", "Linha", "Posicao_Inicio", "Posicao_Fim"])

In [161]:
df = df.sort_values(by=['Linha','Tipo']).reset_index(drop=True)

## Adicionar Coluna Quantidade_Por_Tipo

In [162]:
df["Quantidade"] = df.index + 1
df["Quantidade_Por_Tipo"] = ""

In [163]:
for tipo in df["Tipo"].unique():
    lista_indices = df[df["Tipo"]==tipo].index
    for idx, row in enumerate(lista_indices, start = 1):
        df.loc[row,'Quantidade_Por_Tipo'] = str(idx) + "/" + str(df.loc[row,'Quantidade'])
    

## Arquivo de Saída

In [164]:
nome_arquivo_saida = f'output/{nome_arquivo}.csv'
df.to_csv(nome_arquivo_saida)

In [165]:
df[df["Tipo"]=="Variáveis"]

,Tipo,Palavra,Linha,Posicao_Inicio,Posicao_Fim,Quantidade,Quantidade_Por_Tipo
26,Variáveis,a,8,89,96,27,1/27
41,Variáveis,a,11,133,136,42,2/42


In [166]:
df[df["Tipo"]=="Números Inteiros"]

,Tipo,Palavra,Linha,Posicao_Inicio,Posicao_Fim,Quantidade,Quantidade_Por_Tipo
9,Números Inteiros,1,4,49,50,10,1/10
15,Números Inteiros,1,8,96,97,16,2/16
16,Números Inteiros,1000000,8,101,108,17,3/17
30,Números Inteiros,1,10,130,131,31,4/31
32,Números Inteiros,4,11,143,144,33,5/33
33,Números Inteiros,+1,11,145,147,34,6/34
42,Números Inteiros,1,12,154,155,43,7/43
47,Números Inteiros,2,13,188,189,48,8/48
52,Números Inteiros,3,14,221,222,53,9/53
57,Números Inteiros,4,15,256,257,58,10/58


In [167]:
print(df.to_markdown())

|    | Tipo                | Palavra   |   Linha |   Posicao_Inicio |   Posicao_Fim |   Quantidade | Quantidade_Por_Tipo   |
|---:|:--------------------|:----------|--------:|-----------------:|--------------:|-------------:|:----------------------|
|  0 | Palavras Reservadas | program   |       1 |                0 |             7 |            1 | 1/1                   |
|  1 | Símbolos            | ;         |       1 |               14 |            15 |            2 | 1/2                   |
|  2 | Palavras Reservadas | uses      |       2 |               16 |            20 |            3 | 2/3                   |
|  3 | Símbolos            | ;         |       2 |               24 |            25 |            4 | 2/4                   |
|  4 | Palavras Reservadas | in        |       3 |               34 |            36 |            5 | 3/5                   |
|  5 | Palavras Reservadas | var       |       3 |               26 |            29 |            6 | 4/6                   |
